## YoloV8
- use map50 for accuracy
- quickstart:
  - need to get kaggle key
  - check yaml file
  ```
  train: '/content/sku110k-annotations/SKU110K_fixed_small/images/train'
val: '/content/sku110k-annotations/SKU110K_fixed_small/images/val'
test: '/content/sku110k-annotations/SKU110K_fixed_small/images/test'
# Classes
nc: 1  # number of classes
names: ['object']  # class names
```
  - remember to save model and results folder

```
names: {0: 'object'}
plot: True
results_dict: {'metrics/precision(B)': 0.8677242975920383, 'metrics/recall(B)': 0.7551414103093453, 'metrics/mAP50(B)': 0.8414525640135395, 'metrics/mAP50-95(B)': 0.5051802948639632, 'fitness': 0.5388075217789208}
```

In [5]:
!pip install opendatasets
!pip install pandas
import opendatasets as od
import pandas
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, preprocessing
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Resizing
from tensorflow.keras.models import Sequential

In [ ]:
with tf.device('/device:GPU:0'):
  od.download("https://www.kaggle.com/datasets/thedatasith/sku110k-annotations")

In [6]:
# Install PyTorch if it's not already available (it usually is in Colab)
!pip install torch torchvision
# Install the Ultralytics YOLO package
!pip install ultralytics

In [7]:
import os
import shutil
from random import sample
with tf.device('/device:GPU:0'):
  def create_small_dataset(original_images_dir, original_labels_dir, new_images_dir, new_labels_dir, sample_fraction):
      if not os.path.exists(new_images_dir):
          os.makedirs(new_images_dir)
      if not os.path.exists(new_labels_dir):
          os.makedirs(new_labels_dir)

      # Get all image files in the original image directory
      all_image_files = [f for f in os.listdir(original_images_dir) if f.endswith(('.jpg', '.png'))]
      # Calculate how many files to sample (e.g., 1% of the total)
      num_sampled_files = max(int(len(all_image_files) * sample_fraction), 1)
      # Randomly sample image files
      sampled_image_files = sample(all_image_files, num_sampled_files)

      # Copy the sampled image files and their corresponding label files
      for image_filename in sampled_image_files:
          # Copy image
          shutil.copy(os.path.join(original_images_dir, image_filename), os.path.join(new_images_dir, image_filename))

          # Construct label filename and copy label
          label_filename = image_filename.rsplit('.', 1)[0] + '.txt'
          if os.path.exists(os.path.join(original_labels_dir, label_filename)):
              shutil.copy(os.path.join(original_labels_dir, label_filename), os.path.join(new_labels_dir, label_filename))

  # Paths
  original_train_images = '/content/sku110k-annotations/SKU110K_fixed/images/train'
  original_train_labels = '/content/sku110k-annotations/SKU110K_fixed/labels/train'

  new_train_images = '/content/sku110k-annotations/SKU110K_fixed_small/images/train'
  new_train_labels = '/content/sku110k-annotations/SKU110K_fixed_small/labels/train'

  original_test_images = '/content/sku110k-annotations/SKU110K_fixed/images/test'
  original_test_labels = '/content/sku110k-annotations/SKU110K_fixed/labels/test'

  new_test_images = '/content/sku110k-annotations/SKU110K_fixed_small/images/test'
  new_test_labels = '/content/sku110k-annotations/SKU110K_fixed_small/labels/test'

  original_val_images = '/content/sku110k-annotations/SKU110K_fixed/images/val'
  original_val_labels = '/content/sku110k-annotations/SKU110K_fixed/labels/val'

  new_val_images = '/content/sku110k-annotations/SKU110K_fixed_small/images/val'
  new_val_labels = '/content/sku110k-annotations/SKU110K_fixed_small/labels/val'

  # Create a smaller dataset (1% of the original data)
  create_small_dataset(original_train_images, original_train_labels, new_train_images, new_train_labels, 0.1)
  create_small_dataset(original_test_images, original_test_labels, new_test_images, new_test_labels, 0.1)
  create_small_dataset(original_val_images, original_val_labels, new_val_images, new_val_labels, 0.1)

In [8]:
!nvidia-smi

Thu Nov 30 11:41:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W /  70W |    839MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from ultralytics import YOLO
!pip install torch torchvision
# Install the Ultralytics YOLO package
!pip install ultralytics
import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.0.220 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 43.7/166.8 GB disk)


In [10]:

# Load a pretrained model
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')  # load a pretrained model

# Train the model
model.train(data='/content/sku110k-annotations/data_kaggle.yaml', epochs=10)

# Evaluate model performance on the validation set
metrics = model.val()

# Export the model to ONNX format
path = model.export(format="onnx")


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /content/sku110k-annotations/SKU110K_fixed_small/labels/train... 1557 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1557/1557 [00:02<00:00, 705.10it/s]

train: WARNING ⚠️ /content/sku110k-annotations/SKU110K_fixed_small/images/train/train_1797.jpg: 3 duplicate labels removed


train: New cache created: /content/sku110k-annotations/SKU110K_fixed_small/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/sku110k-annotations/SKU110K_fixed_small/labels/val... 113 images, 0 backgrounds, 0 corrupt: 100%|██████████| 113/113 [00:00<00:00, 613.81it/s]


...


10 epochs completed in 0.150 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.0.220 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [01:33<00:00, 23.31s/it]


                   all        113      17739      0.868      0.753      0.841      0.505
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train3
Ultralytics YOLOv8.0.220 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/sku110k-annotations/SKU110K_fixed_small/labels/val.cache... 113 images, 0 backgrounds, 0 corrupt: 100%|██████████| 113/113 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [01:35<00:00, 11.94s/it]


                   all        113      17739      0.868      0.755      0.841      0.505
Speed: 0.2ms preprocess, 7.6ms inference, 0.0ms loss, 8.3ms postprocess per image
Results saved to runs/detect/train32
Ultralytics YOLOv8.0.220 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.9 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 101.9 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 7.2s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 7.9s, saved as 'runs/detect/train3/weights/best.onnx' (11.7 MB)

Export complete (9.4s)
Results saved to /content/runs/detect/train3/weights
Predict:         y

In [35]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!cp -r "/content/runs/detect/train3" "/content/drive/MyDrive/Term7_cv/yolo_v8" #edit folder name
!cp -r "/content/runs/detect/train32" "/content/drive/MyDrive/Term7_cv/yolo_v8" #edit folder name

In [13]:
metrics = model.val()
print(metrics)

Ultralytics YOLOv8.0.220 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/sku110k-annotations/SKU110K_fixed_small/labels/val.cache... 113 images, 0 backgrounds, 0 corrupt: 100%|██████████| 113/113 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [01:36<00:00, 12.05s/it]


                   all        113      17739      0.868      0.755      0.841      0.505
Speed: 0.2ms preprocess, 4.0ms inference, 0.0ms loss, 7.4ms postprocess per image
Results saved to runs/detect/train33
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b1e7a2af8e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.03

In [27]:
results = model.predict('/content/sku110k-annotations/SKU110K_fixed/images/val/val_472.jpg')


image 1/1 /content/sku110k-annotations/SKU110K_fixed/images/val/val_472.jpg: 640x480 143 objects, 8.9ms
Speed: 2.9ms preprocess, 8.9ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 480)


In [28]:
from PIL import Image
result2 = model('/content/sku110k-annotations/SKU110K_fixed/images/val/val_472.jpg')
for r in result2:
    im_array = r.plot(labels=False)  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('val_472.jpg')


image 1/1 /content/sku110k-annotations/SKU110K_fixed/images/val/val_472.jpg: 640x480 143 objects, 8.7ms
Speed: 2.9ms preprocess, 8.7ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 480)
